In [34]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import numpy as np
import pickle
from matplotlib.patches import Rectangle

In [745]:
try:
    scope.dis()
    target.dis()
except Exception as e:
    pass
    

scope = cw.scope()
target = cw.target(scope)
scope.default_setup()
cw.program_target(scope, cw.programmers.STM32FProgrammer, "./calc-CWNANO.hex")



See https://chipwhisperer.readthedocs.io/en/latest/api.html#firmware-update


Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 13363 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 13363 bytes


In [746]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk


scope.adc.samples = 1500

def reset_target(scope): 
    scope.io.nrst = 'low'
    time.sleep(0.01)
    scope.io.nrst = 'high'
    time.sleep(0.01)

def cap_pass_trace(cmd, msg, rst=True):
    ret = ""
    if rst:
        reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 16)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.simpleserial_write(cmd, msg)

    ret = scope.capture()
    if ret:
        print(f'Timeout happened during acquisition for cmd: {cmd}')

    trace = scope.get_last_trace(as_int=False)
    
    return np.array(trace)


def plot(trace):
    plt.plot(trace)
    plt.show()
    
def sub_plot(trc_arrs):
    plt_no = len(trc_arrs)
    fig, ax = plt.subplots(plt_no,sharex=False)
    for i, obj in enumerate(ax):
        obj.plot(trc_arrs[i])
        obj.set_title(i)
    
        
    plt.show()
    

In [748]:
# {127,-2,1,2,7,6,5,0,4,3}
# markers = ['-ve', '-1', '0', '1', '+ve']
# colors = ['lightblue', 'salmon', 'orange', 'green', 'cyan']

# msg_s = bytearray([1])

msg_i = bytearray([4])

# target.simpleserial_write('s', bytearray([1]))
cap_pass_trace('s', bytearray([0]),True)
trace_i = cap_pass_trace('i', msg_i,False)

# cap_pass_trace('s', bytearray([1]),True)
# trace_m = cap_pass_trace('n', msg_i,False)

# reply = target.simpleserial_read('r',10)
# lis = [int(b) for b in reply]
# print(lis)

# reset_target(scope)
# target.simpleserial_write('q', bytearray([8]))
# cap_pass_trace('s', bytearray([1]))
# trace_m = cap_pass_trace('n', msg_i)


In [749]:
# sub_plot([trace_s, trace_i])
# plt.plot(trace_i)
# plt.plot(trace_m)

# plt.show()


In [750]:
# sub_plot([trace_i, trace_m])

In [751]:
# from numpy import genfromtxt

# pat_ne = trace_i[25:52]
# pat_m1 = trace_i[166:180]
# pat_0 = trace_i[378:393]
# pat_p1 = trace_i[979:999]
# pat_p2 = trace_i[1129:1143]
# pat_pe = trace_i[298:311]

# np.savetxt("pat_0.csv", pat_0, delimiter=",")



In [752]:
from numpy import genfromtxt
pat_ne = genfromtxt('pat_ne.csv', delimiter=',')
pat_m1 = genfromtxt('pat_m1.csv', delimiter=',')
pat_0 = genfromtxt('pat_0.csv', delimiter=',')
pat_p1 = genfromtxt('pat_p1.csv', delimiter=',')
pat_pe = genfromtxt('pat_pe.csv', delimiter=',')

In [753]:
# np.savetxt("pat_ne.csv", pat_ne, delimiter=",")
# np.savetxt("pat_0.csv", pat_0, delimiter=",")
# np.savetxt("pat_p2.csv", pat_p2, delimiter=",")
# np.savetxt("pat_p1.csv", pat_p1, delimiter=",")
# np.savetxt("pat_pe.csv", pat_pe, delimiter=",")

In [754]:
def pattern_match(pat,trace):
    len_p = len(pat)

    l = []

    for i in range(len(trace)-len_p):
        temp = trace[i:i+len_p]
        cf = np.corrcoef(pat,temp)[0][1]
        l.append(cf)
    return l

In [755]:
patterns = [pat_ne,pat_0,pat_p1,pat_p2,pat_pe]

found_patterns = []

for i, pat in enumerate(patterns):
    found_patterns.append(pattern_match(pat, trace_i))
    

In [756]:
# sorted(found_patterns[4])[::-1]

In [761]:
res = []
markers = ['-ve', '0', '1','2', '+ve']

for i in range(len(found_patterns)):
    
    for j, cv in enumerate(found_patterns[i]):
        if cv > .95:
            res.append((j,markers[i], cv))

In [760]:
sorted(res)

[(298, '-ve', 0.9836913229337555),
 (585, '+ve', 0.9568788480669652),
 (719, '-ve', 0.9796707617712695)]

In [762]:
%matplotlib tk
from matplotlib.patches import Rectangle

# markers = ['-ve', '0' ,1', '2', '+ve']
colors = ['lightblue', 'salmon', 'orange', 'lime', 'cyan']

fig, ax = plt.subplots(2,sharex=False)
ax[0].plot(trace_i)
for i in res:
    index = i[0]
    marker = i[1]
    marker_index = markers.index(marker)
    pat_len = len(patterns[marker_index])
    
    rect = Rectangle((index, -0.4), pat_len, 1, facecolor=colors[marker_index])
    ax[0].add_patch(rect)
    
ax[1].plot(pat_0)
plt.show()

# fig, ax = plt.subplots(2,sharex=False)
# ax[0].plot(trace_i)
# for i in res:
#     index = i[0]
#     marker = i[1]
#     marker_index = markers.index(marker)
#     pat_len = len(patterns[marker_index])
    
#     rect = Rectangle((index, -0.4), pat_len, 1, facecolor=colors[marker_index])
#     ax[0].add_patch(rect)
    
# ax[1].plot(trace_m)
# plt.show()

In [685]:
import math
math.ceil(300/255) 1.25

2

In [707]:
from tqdm.notebook import tqdm

In [730]:
from math import ceil 
scalar = bytearray([4])

for i in tqdm(range(410,490)):
    for j in range(ceil(i/255)):
        if j == ceil(i/255)-1:
            msg = i%255
            if msg == 0:
                msg = 255
        else :
            msg = 255
        cap_pass_trace('p', bytearray([msg]),j==0)
            
    trace = cap_pass_trace('i', scalar,False)
    mch = pattern_match(pat_p1,trace)
    num_0 = 0
    for m in mch:
        if m>0.98:
            num_0 +=1
    if num_0 :
        print(f"{1-i} found ")
    
for i in tqdm(range(2**32)):
    for j in range(ceil(i/255)):
        if j == ceil(i/255)-1:
            msg = i%255
            if msg == 0:
                msg = 255
        else :
            msg = 255
        cap_pass_trace('s', bytearray([msg]),j==0)
    
    trace = cap_pass_trace('i', scalar,False)
    mch = pattern_match(pat_p1,trace)
    num_0 = 0
    for m in mch:
        if m>0.95:
            num_0 +=1
    if num_0 :
        print(f"{i+1} found")

  0%|          | 0/4294967296 [00:00<?, ?it/s]

65 found


KeyboardInterrupt: 

In [ ]:
[-3,-2,-1,0,1]
[-2,-1,0,1,2] 0 found


In [194]:
# target.simpleserial_write('q', bytearray([8]))
# cap_pass_trace('s', bytearray([2]))
# trace = cap_pass_trace('i', scalar)
# mch = pattern_match(pat_0,trace)
# num_0 = 0
# for m in mch:
#     if m>0.9:
#         num_0 +=1
# print(f"{2}\tnumber of zeros found {num_0}")

2	number of zeros found 1


In [196]:
# plt.plot(trace)
# plt.show()

In [240]:
tr = trace_i.copy()

In [253]:
tr

array([0.09765625, 0.18359375, 0.203125  , ..., 0.18359375, 0.22265625,
       0.49609375])

In [254]:
trace_i

array([0.1015625 , 0.18359375, 0.203125  , ..., 0.1875    , 0.203125  ,
       0.49609375])